In [1]:
# from pytest import MonkeyPatch
from pathlib import Path
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool
from process_analyst_copilot import ClarifyTheAsk
from process_analyst_copilot.SemanticAssert import semantic_assert


def clarify_the_ask() -> ClarifyTheAsk:

    # # OpenAI setup for pytest
    # from dotenv import load_dotenv, find_dotenv

    # load_dotenv(find_dotenv())
    # return ClarifyTheAsk()

    # Ollama setup for pytest
    from process_analyst_copilot import OllamaLLM

    llm_model = OllamaLLM(
        model="ollama/llama3.1:8b",
        temperature=0.3,
        api_base="http://localhost:11434",
    )
    # Ollama default context window
    # 2048 + 1 to trigger OllamaLLM context window warning
    llm_model.num_ctx = 16384
    x = ClarifyTheAsk(llm_model=llm_model)
    x.embedder = {"provider": "ollama", "config": {"model": "llama3.1:8b"}}
    return x

In [2]:
# def test_llm_response(clarify_the_ask: ClarifyTheAsk) -> None:
#     # Given some input data
#     input_data = "In one word what is the colour of the sky?"
#     expected_output = "blue"

#     # When calling your agent's method or task processing logic
#     result: str = clarify_the_ask.test_llm(content=input_data)
#     print(result)
#     # Then assert that the result meets your expected output
#     assert semantic_assert(
#         expected_output, result
#     ), f"Expected {expected_output}, but got {result}"


# test_llm_response(clarify_the_ask())

In [6]:
# def test_bpa_agent_response(clarify_the_ask: ClarifyTheAsk) -> None:
#     clarify_the_ask.setup_bpa_agent()

#     # Given some input data
#     input_data = "What is the first step to improve an existing process?"
#     expected_output = "Requirement gathering"

#     # When calling your agent's method or task processing logic
#     result: str = clarify_the_ask.business_process_analyst.execute_task(
#         Task(
#             description=input_data,
#             expected_output="In two words only",
#             max_retries=0,
#         )
#     )
#     print(result)
#     # Then assert that the result meets your expected output
#     assert semantic_assert(
#         expected_output, result
#     ), f"Expected {expected_output}, but got {result}"


# test_bpa_agent_response(clarify_the_ask())

In [7]:
def test_pqa_agent_response(clarify_the_ask: ClarifyTheAsk) -> None:
    clarify_the_ask.setup_pqa_agent()

    # Given some input data
    input_data = (
        "In two words only, what is the primary goal of CQPA process improvement?"
    )
    expected_output = "Process Enhancement"

    # When calling your agent's method or task processing logic
    result: str = clarify_the_ask.process_analyst_quality_assurance.execute_task(
        Task(
            description=input_data,
            expected_output="Two words only",
            max_retries=0,
        )
    )
    print(result)
    # Then assert that the result meets your expected output
    assert semantic_assert(
        expected_output, result
    ), f"Expected {expected_output}, but got {result}"


test_pqa_agent_response(clarify_the_ask())

h:\Python\AI Projects\process-analyst-copilot\.venv\Lib\site-packages\pypdf\generic\_data_structures.py:1467: ResourceWarning: unclosed <socket.socket fd=4980, family=2, type=1, proto=0, laddr=('127.0.0.1', 63024), raddr=('127.0.0.1', 11434)>
  if IndirectPattern.match(peek) is not None:
h:\Python\AI Projects\process-analyst-copilot\.venv\Lib\site-packages\pypdf\generic\_data_structures.py:1467: ResourceWarning: unclosed <socket.socket fd=5104, family=2, type=1, proto=0, laddr=('127.0.0.1', 63027), raddr=('127.0.0.1', 11434)>
  if IndirectPattern.match(peek) is not None:


# Agent: Process Analyst Quality Assurance (CPQA Certified)
## Task: In two words only, what is the primary goal of CQPA process improvement?
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query the H:/Python/AI Projects/process-analyst-copilot/config/references/certified-quality-process-analyst-handbook.pdf PDF's content.



# Agent: Process Analyst Quality Assurance (CPQA Certified)
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\

AssertionError: Expected Process Enhancement, but got The primary goal of CQPA (Certified Quality Process Analyst) process improvement is to enhance the quality of processes within an organization by identifying areas for improvement, implementing changes, and monitoring results. This involves a systematic approach to analyze and optimize business processes to increase efficiency, reduce costs, and improve overall performance.

CQPA process improvement typically involves:

1.  **Process Mapping**: Creating visual representations of current processes to identify inefficiencies and opportunities for improvement.
2.  **Root Cause Analysis**: Identifying the underlying causes of problems or inefficiencies in processes.
3.  **Process Redesign**: Designing new, improved processes that address identified issues and improve overall performance.
4.  **Implementation**: Putting new processes into place and ensuring they are adopted by all relevant stakeholders.
5.  **Monitoring and Evaluation**:

In [ ]:
# Example test case for `draft_process`
def test_draft_process(clarify_the_ask: ClarifyTheAsk) -> None:
    clarify_the_ask.setup_bpa_agent()
    clarify_the_ask.setup_draft_process()

    # Given some input data
    input_ask = "The simplest way to make a cup of tea?"
    expected_output = """
        1. Boil water
        2. Add tea leaves or tea bag to cup
        3. Pour hot water into the cup
        4. Steep for desired time
        5. Remove tea bag or leaves
        6. Serve
    """

    clarify_the_ask.draft_process.output_file = None
    crew = Crew(
        agents=[clarify_the_ask.business_process_analyst],
        tasks=[clarify_the_ask.draft_process],
    )
    result: str = crew.kickoff(
        inputs={
            "input_ask": input_ask,
        }
    ).raw
    print(result)
    assert semantic_assert(
        expected_output, result
    ), f"Expected {expected_output}, but got {result}"